In [ ]:
import pandas as pd
from pprint import pprint
from pathlib import Path
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from collections import defaultdict
import mplhep as hep
import datetime
hep.style.use('CMS')

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# import os, sys

# path2add = os.path.normpath(os.path.abspath(os.path.join(os.path.curdir, os.path.pardir, 'TestBeam')))
# print(path2add)

# if (not (path2add in sys.path)) :
#     sys.path.append(path2add)

# from beamtest_analysis_helper import plot_BL_and_NW

# path2add = os.path.normpath(os.path.abspath(os.path.join(os.path.curdir, os.path.pardir, 'SEU')))
# print(path2add)

# if (not (path2add in sys.path)) :
#     sys.path.append(path2add)

# from SEUhelper import plotVRefPower,plotBoardPower,plotWSPower

## Baseline plots

In [ ]:
con = sqlite3.connect('/home/daq/ETROC2/ETROC-History/CERN_Aug_2024_TID_history/BaselineHistory.sqlite')

baseline_df = pd.read_sql_query("SELECT * from baselines", con)

baseline_df['timestamp'] = pd.to_datetime(baseline_df['timestamp'], format='%Y-%m-%d %H:%M:%S.%f')
baseline_df['timestamp'] = baseline_df['timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')
baseline_df['timestamp'] = pd.to_datetime(baseline_df['timestamp'])

In [ ]:
baseline_df.drop(columns=['save_notes'], inplace=True)

In [ ]:
### Add board names in the dictionary

config_dict = {
    'ET2p03_BARE7': {
        'plot_title': 'ETROC 2.03 Bare Board 7',
        'chip_type': 'T',
        'channel': 0,
    },
}

### Baseline over time

In [ ]:
# p_col=[14,14,14,0,0,0]
# p_row=[0,3,12, 0,3,12]
p_col = [7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8]
p_row = [15,14,13,12,3,2,1,0, 15,14,13,12,3,2,1,0]
pixel_plot_list = list(zip(p_row,p_col))
colors = {(row,col):'r' if row in [0,1,2,3] else 'b' for row,col in pixel_plot_list}

draw_dates = True
# cut_time = pd.to_datetime('2024-08-03 16:00:00')
# cut_time = pd.to_datetime('2024-08-07 2:00:00')
# cut_time = pd.to_datetime('2024-08-08 17:00:00')
# cut_time = pd.to_datetime('2024-08-10 20:00:00')
# cut_time = pd.to_datetime('2024-08-11 14:50:00')

cut_time = pd.to_datetime('2024-08-13 20:30:00')

for iboard in baseline_df['chip_name'].unique():
    if iboard not in config_dict: continue
    fig, ax = plt.subplots(figsize=(20, 10))
    hep.cms.text(loc=0, ax=ax, text="ETL ETROC SEU Testing", fontsize=25)
    ax.set_title(f'Baseline of {config_dict[iboard]["plot_title"]}', size=17, loc="right", fontsize=25)
    date_form = mdates.DateFormatter("%m-%d\n%H:%M")
    ax.xaxis.set_major_formatter(date_form)
    for row,col in pixel_plot_list:
        interest_df = baseline_df.loc[baseline_df['chip_name']==iboard]
        tmp_df = interest_df.loc[(interest_df['col']==col) & (interest_df['row']==row)].reset_index(drop=True)
        tmp_df = tmp_df.loc[tmp_df['timestamp'] > cut_time]
        ax.plot(tmp_df['timestamp'], tmp_df['baseline'], label=f'Pixel R{row} C{col}', marker='.',color=colors[(row,col)])
    ax.set_xlim(left=cut_time)
    # ax.legend()

## Temperature

In [ ]:
con = sqlite3.connect('/home/daq/ETROC2/ETROC-History/CERN_Aug_2024_TID_history/ADCHistory.sqlite')
adc_df = pd.read_sql_query("SELECT * from adc", con)

In [ ]:
adc_df['timestamp'] = pd.to_datetime(adc_df['timestamp'], format='%Y-%m-%d %H:%M:%S.%f')
adc_df['timestamp'] = adc_df['timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')
adc_df['timestamp'] = pd.to_datetime(adc_df['timestamp'])

In [ ]:
adc_df.tail()

In [ ]:
# Start: 23:48 on Aug 4th,  2024
# Stop : 02:46 on Aug 7th,  2024
# Start: 17:23 on Aug 8th,  2024
# Stop : 20:22 on Aug 10th, 2024



In [ ]:
# cut_time = pd.to_datetime('2024-08-04 23:00:00')
# cut_time = pd.to_datetime('2024-08-07 2:00:00')
# cut_time = pd.to_datetime('2024-08-08 17:00:00')
# cut_time = pd.to_datetime('2024-08-10 20:00:00')

# cut_time = pd.to_datetime('2024-08-13 20:30:00')

rcut_time = None

# cut_time = pd.to_datetime('2024-08-04 23:48:00')
# rcut_time = pd.to_datetime('2024-08-07 2:46:00')

# cut_time = pd.to_datetime('2024-08-08 17:23:00')
# rcut_time = pd.to_datetime('2024-08-10 20:22:00')

fig, ax = plt.subplots(figsize=(20, 10))
date_form = mdates.DateFormatter("%m-%d\n%H:%M")
ax.xaxis.set_major_formatter(date_form)

tmp_df = adc_df.loc[adc_df['channel']==7]
ax.plot(tmp_df['timestamp'], tmp_df['voltage'], label=f'VTemp')
ax.grid(which="both")
ax.legend(loc='best')
ax.set_ylabel('VTemp Voltage [V]')
ax.set_ylim(0.42, 0.575)
ax.set_xlim(left=cut_time)
if(rcut_time!=None): ax.set_xlim(left=cut_time, right=rcut_time)

In [ ]:
# names = ['BARE BOARD 7', 'Ambient Temp']
# cut_time = pd.to_datetime('2024-06-24 22:00:00')

# fig, ax = plt.subplots(figsize=(20, 10))
# hep.cms.text(loc=0, ax=ax, text="Phase-2 Preliminary", fontsize=25)
# ax.set_title('Temperature', size=17, loc="right")
# date_form = mdates.DateFormatter("%m-%d\n%H:%M")
# ax.xaxis.set_major_formatter(date_form)

# for idx, iboard in enumerate(sorted(adc_df['channel'].unique())):
#     tmp_df = adc_df.loc[adc_df['channel']==iboard]
#     ax.plot(tmp_df['timestamp'], tmp_df['voltage'], label=f'{names[idx]}', marker=".", ls="")

# text_y = [0.51, 0.55]
# if draw_dates:
#         for index,run_dict in enumerate(run_info):
#             if index%2 == 0:
#                 yval = text_y[0]
#             else:
#                 yval = text_y[1]
#             if(run_dict['start'] >= cut_time):
#                 ax.axvline(x=run_dict['start'], color='magenta', linestyle='dashed')
#                 ax.text(x=(run_dict['start'] ), y=yval, s=f"{run_dict['name']} Start", rotation=90, va='center', fontsize=12, color='magenta')
#                 ax.axvline(x=run_dict['stop'], color='black', linestyle='dashdot')
#                 ax.text(x=(run_dict['stop'] ), y=yval, s=f"{run_dict['name']} End", rotation=90, va='center', fontsize=12)
                
# ax.legend(loc='best')
# ax.set_ylabel('Voltage')
# plt.ylim(bottom=0.49, top=.56)
# plt.xlim(left=cut_time)

# Power Monitoring

In [ ]:
start_time = datetime.datetime(2024, 8, 13, 20, 30, 0)
# end_time = datetime.datetime(2024, 6, 25, 19, 0, 0)

power_file = Path("/home/daq/ETROC2/ETROC-History/CERN_Aug_2024_TID_history/NEW_PowerHistory_v2.sqlite")

power_connections = {
    "ET2.03 Bare Board 7": {
        "Analog": "Analog",
        "Digital": "Digital",
    },
}

In [ ]:
# power_directory = output_dir/"PowerPlots"
# power_directory.mkdir(exist_ok=True)

# cut_time = pd.to_datetime('2024-08-04 23:00:00')
# cut_time = pd.to_datetime('2024-08-07 2:00:00')
# cut_time = pd.to_datetime('2024-08-08 17:00:00')
# cut_time = pd.to_datetime('2024-08-12 16:50:00')


with sqlite3.connect(power_file) as sqlite3_connection:
    data_df = pd.read_sql('SELECT * FROM power_v2', sqlite3_connection, index_col=None)
    data_df['Time'] = pd.to_datetime(data_df['timestamp'], format='mixed')

    # Remove data that is outside the range of the campaign
    data_df = data_df.loc[data_df['Time'] >= start_time]
    # data_df = data_df.loc[data_df['Time'] <= end_time].copy()

    # print(data_df)
    data_df['V'] = (data_df['V'].str.replace('V','')).astype(float)
    data_df['I'] = (data_df['I'].str.replace('A','')).astype(float)

    print("Timestamps in power database file cover range:")
    print("Min:", data_df['Time'].min())
    print("Max", data_df['Time'].max())

    tmp_df = data_df.loc[data_df['Instrument'] == 'Power']   
    tmp_df = tmp_df.loc[tmp_df['Channel'] == 'Digital']
    
    fig, axes = plt.subplots(2, 1, figsize=(20, 12))
    date_form = mdates.DateFormatter("%m-%d\n%H:%M")
    axes[0].set_title('ET2 Digital power', loc='right')
    # axes[0].xaxis.set_major_formatter(date_form)
    axes[0].xaxis.set_ticklabels([])
    axes[0].plot(tmp_df['Time'], tmp_df['V'])
    axes[0].set_ylabel('Voltage [V]')

    axes[1].xaxis.set_major_formatter(date_form)
    axes[1].plot(tmp_df['Time'], tmp_df['I'])
    axes[1].set_ylabel('Currents [A]')

    # axes[0].set_ylim(0.9, 1.4)
    # axes[1].set_ylim(0.075, 0.15)
    axes[0].set_xlim(left=cut_time)
    axes[1].set_xlim(left=cut_time)

    plt.tight_layout()

In [ ]:
# power_directory = output_dir/"PowerPlots"
# power_directory.mkdir(exist_ok=True)

with sqlite3.connect(power_file) as sqlite3_connection:
    data_df = pd.read_sql('SELECT * FROM power_v2', sqlite3_connection, index_col=None)
    data_df['Time'] = pd.to_datetime(data_df['timestamp'], format='mixed')

    # Remove data that is outside the range of the campaign
    data_df = data_df.loc[data_df['Time'] >= start_time]
    # data_df = data_df.loc[data_df['Time'] <= end_time].copy()

    data_df['V'] = (data_df['V'].str.replace('V','')).astype(float)
    data_df['I'] = (data_df['I'].str.replace('A','')).astype(float)

    print("Timestamps in power database file cover range:")
    print("Min:", data_df['Time'].min())
    print("Max", data_df['Time'].max())

    tmp_df = data_df.loc[data_df['Instrument'] == 'Power']   
    tmp_df = tmp_df.loc[tmp_df['Channel'] == 'Analog']
    
    fig, axes = plt.subplots(2, 1, figsize=(20, 12))
    date_form = mdates.DateFormatter("%m-%d\n%H:%M")
    axes[0].set_title('ET2 Analog power', loc='right')
    # axes[0].xaxis.set_major_formatter(date_form)
    axes[0].xaxis.set_ticklabels([])
    axes[0].plot(tmp_df['Time'], tmp_df['V'])
    axes[0].set_ylabel('Voltage [V]')
    
    axes[1].xaxis.set_major_formatter(date_form)
    axes[1].plot(tmp_df['Time'], tmp_df['I'])
    axes[1].set_ylabel('Currents [A]')

    # axes[0].set_ylim(0.9, 1.5)
    axes[0].set_xlim(left=cut_time)
    axes[1].set_xlim(left=cut_time)

    plt.tight_layout()

## Waveform Sampler

In [ ]:
# power_directory = output_dir/"PowerPlots"
# power_directory.mkdir(exist_ok=True)

# power_file = Path("/home/daq/ETROC2/ETROC-History/CERN_Aug_2024_TID_history/WS_PowerHistory_v2.sqlite")

with sqlite3.connect(power_file) as sqlite3_connection:
    data_df = pd.read_sql('SELECT * FROM power_v2', sqlite3_connection, index_col=None)
    data_df['Time'] = pd.to_datetime(data_df['timestamp'], format='mixed')

    # Remove data that is outside the range of the campaign
    data_df = data_df.loc[data_df['Time'] >= start_time]
    # data_df = data_df.loc[data_df['Time'] <= end_time].copy()

    data_df['V'] = (data_df['V'].str.replace('V','')).astype(float)
    data_df['I'] = (data_df['I'].str.replace('A','')).astype(float)

    print("Timestamps in power database file cover range:")
    print("Min:", data_df['Time'].min())
    print("Max", data_df['Time'].max())

    tmp_df = data_df.loc[data_df['Instrument'] == 'WS Power']   
    tmp_df = tmp_df.loc[tmp_df['Channel'] == 'Digital']
    
    fig, axes = plt.subplots(2, 1, figsize=(20, 12))
    date_form = mdates.DateFormatter("%m-%d\n%H:%M")
    axes[0].set_title('ET2 WS Digital power', loc='right')
    # axes[0].xaxis.set_major_formatter(date_form)
    axes[0].xaxis.set_ticklabels([])
    axes[0].plot(tmp_df['Time'], tmp_df['V'])
    axes[0].set_ylabel('Voltage [V]')

    axes[1].xaxis.set_major_formatter(date_form)
    axes[1].plot(tmp_df['Time'], tmp_df['I'])
    axes[1].set_ylabel('Currents [A]')

    # axes[0].set_ylim(0.9, 1.5)
    axes[0].set_xlim(left=cut_time)
    axes[1].set_xlim(left=cut_time)

    plt.tight_layout()

In [ ]:
# power_directory = output_dir/"PowerPlots"
# power_directory.mkdir(exist_ok=True)

with sqlite3.connect(power_file) as sqlite3_connection:
    data_df = pd.read_sql('SELECT * FROM power_v2', sqlite3_connection, index_col=None)
    data_df['Time'] = pd.to_datetime(data_df['timestamp'], format='mixed')

    # Remove data that is outside the range of the campaign
    # data_df = data_df.loc[data_df['Time'] >= start_time]
    # data_df = data_df.loc[data_df['Time'] <= end_time].copy()

    data_df['V'] = (data_df['V'].str.replace('V','')).astype(float)
    data_df['I'] = (data_df['I'].str.replace('A','')).astype(float)

    print("Timestamps in power database file cover range:")
    print("Min:", data_df['Time'].min())
    print("Max", data_df['Time'].max())

    tmp_df = data_df.loc[data_df['Instrument'] == 'WS Power']   
    tmp_df = tmp_df.loc[tmp_df['Channel'] == 'Analog']
    
    fig, axes = plt.subplots(2, 1, figsize=(20, 12))
    date_form = mdates.DateFormatter("%m-%d\n%H:%M")
    axes[0].set_title('ET2 WS Analog power', loc='right')
    # axes[0].xaxis.set_major_formatter(date_form)
    axes[0].xaxis.set_ticklabels([])
    axes[0].plot(tmp_df['Time'], tmp_df['V'])
    axes[0].set_ylabel('Voltage [V]')
    # axes[0].set_ylim(0.9, 1.5)
    axes[1].xaxis.set_major_formatter(date_form)
    axes[1].plot(tmp_df['Time'], tmp_df['I'])
    axes[1].set_ylabel('Currents [A]')

    # axes[0].set_xlim(left=cut_time)
    # axes[1].set_xlim(left=cut_time)

    plt.tight_layout()

## VRef

In [ ]:
con = sqlite3.connect('/home/daq/ETROC2/ETROC-History/CERN_Aug_2024_TID_history/vol_monitoring.sqlite')
vol_df = pd.read_sql_query("SELECT * from vol_monitoring", con)

# adc_df['timestamp'] = pd.to_datetime(adc_df['timestamp'], format='%Y-%m-%d %H:%M:%S.%f')
# adc_df['timestamp'] = adc_df['timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')
vol_df['timestamp'] = pd.to_datetime(vol_df['timestamp'])

In [ ]:
# cut_time = pd.to_datetime('2024-08-04 23:00:00')
# cut_time = pd.to_datetime('2024-08-07 2:00:00')
# cut_time = pd.to_datetime('2024-08-08 17:00:00')
# cut_time = pd.to_datetime('2024-08-10 20:00:00')

rcut_time = None

# cut_time = pd.to_datetime('2024-08-04 23:48:00')
# rcut_time = pd.to_datetime('2024-08-07 2:46:00')

# cut_time = pd.to_datetime('2024-08-08 17:23:00')
# rcut_time = pd.to_datetime('2024-08-10 20:22:00')

lines = [pd.to_datetime('2024-08-04 23:48:00'), pd.to_datetime('2024-08-07 2:46:00'), pd.to_datetime('2024-08-08 17:23:00'), pd.to_datetime('2024-08-10 20:22:00')]

fig, ax = plt.subplots(figsize=(20, 10))
date_form = mdates.DateFormatter("%m-%d\n%H:%M")
ax.xaxis.set_major_formatter(date_form)

ax.plot(vol_df['timestamp'], vol_df['voltage'], label=f'VRef')

ax.legend(loc='best')
ax.set_ylabel('VRef Voltage [V]')
ax.set_ylim(0.84, None)
ax.set_xlim(left=cut_time)
if(rcut_time!=None): ax.set_xlim(left=cut_time, right=rcut_time)

ax.vlines(x=lines,ymin=0, ymax=1, color='r',ls='--', lw=2)